In [ ]:
# Transformers installation
! pip install transformers
! pip install transformers datasets
! pip install sentencepiece
! pip install sacremoses
! pip install datasets evaluate transformers[sentencepiece]
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer
from datasets import Dataset
import pandas as pd
df = pd.read_csv("21-0061.csv", encoding="cp1252", index_col=0)  # or load from pandas DataFrame
sentences = df.iloc[0:2, 0].tolist()
labels = df.iloc[0:2, 1].tolist()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
data_dict = {"sentence": sentences, "label": labels}
dataset = Dataset.from_dict(data_dict)
print(dataset)

Dataset({
    features: ['sentence', 'label'],
    num_rows: 2
})


In [ ]:
def tokenize_function(batch):
    return tokenizer(batch["sentence"], padding="max_length", truncation=True)

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
print(tokenized_dataset)

Dataset({
    features: ['sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2
})


In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, test_dataset = train_test_split(tokenized_dataset, test_size=0.2)
print(train_dataset)
print(test_dataset)

{'label': tensor([0.]), 'input_ids': tensor([[  101,   155,  9919, 12420, 10036,  3048,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

KeyError: ignored